In [ ]:
import porespy as ps
import h5py
import pandas as pd
import numpy as np
from tqdm import tqdm

f = "Blunt_images.h5"

with h5py.File(f) as data:
    keys = [k for k in data if "image" in k]

    for k in tqdm(keys):
        all_data = []
        if "image" in k:
            print(k)
            im = np.array(data[k])

            for size in tqdm([10,13,16,20,25,33,50,100]):

                s = ps.tools.get_slices_random(im, n=100, lims=[size,size])

                df = ps.metrics.rev_tortuosity(im=im, slices=s)
                tmp = ps.tools.results_to_df(df)
                all_data.append(tmp)

            all_data_df = pd.concat(all_data)
            all_data_df.to_csv(f"Blunt_REV_all_same_blocks/{k}.csv")

            # tmp.to_csv(f"Blunt_REV_random_blocks/{k}.csv")
        
            

In [ ]:
import h5py
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import Normalize, LogNorm
import porespy as ps
from scipy.optimize import fmin
from sklearn.metrics import r2_score
from scipy.stats import chi2

# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.serif'] = ['Times New Roman']
all_mask = True
cmap = cm.viridis
colours = ['C0', 'C1', 'C2', 'C3', ]


block_size_universal = [
    10,
    13,
    16,
    20,
    25,
    33,
    50,
    100,
]

def hex_plot(df, im_size, ep_thresh, fig=None, axes=None, bounds=None, actual_tau=None,):

    for unique_axis in np.unique(df['axis']):
        if axes != None:

            plt.sca(axes[unique_axis])

        else:
            fig, ax = plt.subplots(dpi=300)

        tmp = df[df['axis']==unique_axis]

        mask1 = tmp['tau'] != 0
        mask2 = tmp['tau'] != np.inf
        mask3 = tmp['eps_perc'] < ep_thresh*1.1
        mask4 = tmp['eps_perc'] > ep_thresh*0.9

        tmp = tmp[np.logical_and(
            mask1,
            mask2,
            # mask3,
            # mask4,
            )]


        ny = 12
        nx=int(np.sqrt(3) * ny)

        vol_frac = np.log10(tmp['length']**3 / (im_size **3))

        bin1 = plt.hexbin(
            vol_frac,
            1 / tmp['tau'],
            gridsize = [nx, ny],
            cmap='hot',
            mincnt=1,
            edgecolors='red',
            norm=LogNorm(vmin=1, vmax=1000)

        )
        plt.xlabel(fr"$log_{{10}}(\phi)$")
        plt.ylabel(fr'$1 / \tau$')
        plt.title(f'Tau Distribution - Axis {unique_axis}')
        plt.ylim([0, 1])

        if unique_axis == 2 and axes!=None:
            cb = plt.colorbar(bin1)
            cb.set_label(fr'Frequency of $\tau$')
        
        else:
            cb = plt.colorbar(bin1)
            cb.set_label(fr'Frequency of $\tau$')

        if bounds and type(actual_tau) == np.ndarray:
            # plt.plot(vol_frac, np.array(mean_vals) * 1.2, '--k', alpha=0.5)
            # plt.plot(vol_frac, np.array(mean_vals) * 0.8, '--k', alpha=0.5)
            plt.plot([min(vol_frac), max(vol_frac)], [(1 / actual_tau[unique_axis]) * 1.2] * 2, '--k', alpha=0.5)
            plt.plot([min(vol_frac), max(vol_frac)], [(1 / actual_tau[unique_axis]) * 0.8] * 2, '--k', alpha=0.5)


    if axes != None and len(axes) == 4:
        axes[3].clear()
        plt.sca(axes[3])
    
    else:
        fig, ax = plt.subplots(dpi=300)
    bin2 = plt.hexbin(
        np.log10(tmp['length']**3 / (im_size **3)),
        tmp['eps_perc'],
        gridsize = [nx, ny],
        cmap='viridis',
        mincnt=1,
        edgecolors='red',
        norm=LogNorm(vmin=1, vmax=1000)

    )
    plt.xlabel(fr"$log_{{10}}(\phi)$")
    plt.ylabel(fr'$\varepsilon$')
    plt.title(f'Porosity Distribution')
    plt.ylim([0, 1])

    cb = plt.colorbar(bin2)
    cb.set_label(fr'Frequency of $\varepsilon$')

    plt.show()

def rev_plot(df, size, im_n, title="", ep_thresh=0.5, figs=None, axes=None, bounds=False, actual_tau=None):
    plot_size = 4

    valid_tau_values = {}

    if figs == None or axes == None:
        # figs, axes = plt.subplots(ncols=3, figsize=[4*3, 4])
        f1, ax1 = plt.subplots(dpi=300)
        f2, ax2 = plt.subplots(dpi=300)
        f3, ax3 = plt.subplots(dpi=300)
        figs = [f1,f2,f3]
        axes = [ax1,ax2,ax3]
    

    for i, axis in enumerate(np.unique(df['axis'])):
        tmp = df[df['axis']==axis]

        log_tau = []
        vol_frac = []
        mean_vals = []
        st_dev = []

        tmp_dict = {}

        # for vol in np.unique(tmp['volume']):
        for uu in range(1,5):
            tmp_valid = []
            for vol in np.array(block_size_universal)**3:
                tmp2 = tmp[tmp['volume']==vol]["tau"]
                taus = np.copy(tmp2)
                # eps = tmp[tmp['volume']==vol]['porosity_perc']
                eps = tmp[tmp['volume']==vol]['porosity_perc']
                # second_largest = np.unique(tmp2)[-2]
                # taus[tmp2==np.inf] = second_largest
                taus = 1/taus
                mask = taus!=0
                mask2 = eps < ep_thresh * (1 + uu * 0.05)
                mask3 = eps > ep_thresh * (1 - uu * 0.05)

                if all_mask and uu == 4:
                    log_tau.append(taus[mask * mask2 * mask3])
                    mean_vals.append(np.mean(taus[mask * mask2 * mask3]))
                    st_dev.append(np.std(tmp2[mask*mask2*mask3]))


                elif uu==4:
                    log_tau.append(taus[mask])
                    mean_vals.append(np.mean(taus[mask]))
                    st_dev.append(np.std(tmp2[mask]))
            

                if uu==4:
                    vol_frac.append(np.log10(vol / (size**3)))
                
                if all_mask:
                    tmp_valid.append(len(taus[mask * mask2 * mask3]) / len(taus))
                
                else:
                    tmp_valid.append(len(taus[mask]) / len(taus))

            tmp_dict[uu] = tmp_valid
        
        valid_tau_values[i] = tmp_dict
        
        plt.sca(axes[i])

        norm = Normalize(vmin=0, vmax=len(log_tau)-1)
        parts = plt.violinplot(log_tau, vol_frac, widths=0.1)
        for item in ['cbars', 'cmins', 'cmaxes']:
            parts[item].set_color('k')

        for qq in range(1,5):

            plt.plot([min(vol_frac), max(vol_frac)], [(1 / actual_tau[i]) * (1 + qq*0.05)] * 2, f'--{colours[qq-1]}')
            plt.plot([min(vol_frac), max(vol_frac)], [(1 / actual_tau[i]) * (1 - qq*0.05)] * 2, f'--{colours[qq-1]}')

        for j, body in enumerate(parts['bodies']):
            color = cmap(norm(j))
            body.set_facecolor(color)
            body.set_edgecolor(color)
            body.set_alpha(0.7)
            m = np.mean(body.get_paths()[0].vertices[:, 0])  # middle position
            verts = body.get_paths()[0].vertices
            verts[:, 0] = np.clip(verts[:, 0], m, np.inf)

        plt.plot(vol_frac, mean_vals, 'k')
        plt.title(f"REV: Axis {axis} : {title}")
        plt.xlabel(r"$log_{10} (\frac{V_{B}}{V_{T}})$")
        plt.ylabel(r"$\frac{1}{\tau}$")
        plt.tight_layout()
        plt.savefig(fr"Figures/rev_tau_{im_n}_{axis}.png")

        

    return figs, axes, mean_vals, st_dev, log_tau, valid_tau_values

def analyze_rev_poro(poro_df, p_compare, ax, im_n, im_size, ep_thresh=None):
    plt.sca(ax)

    poro_check = {}
    valid_frac = {}
    # ==========POROSITY CHECK==========

    all_porosity = []

    for vol in np.unique(poro_df['volume']):
        all_porosity.append(poro_df['porosity'][poro_df['volume']==vol])

    plt.plot(np.log10(poro_df['volume'] / im_size), poro_df['porosity'], '.k', alpha=0.5)
    # plt.violinplot(all_porosity, np.log10(np.unique(poro_df['volume']) / im_size))

    for ii in range(1,5):
        upper_bound = p_compare * (1 + ii * 0.05)
        lower_bound = p_compare * (1 - ii * 0.05)

        plt.plot([np.log10(min(poro_df['volume']) / im_size), np.log10(max(poro_df['volume']) / im_size)],
                  [lower_bound]*2,
                  f'--{colours[ii-1]}',
                  label=fr"$\pm${ii*0.05:.2f}")
        
        plt.plot([np.log10(min(poro_df['volume']) / im_size), np.log10(max(poro_df['volume']) / im_size)], 
                 [upper_bound]*2,
                 f'--{colours[ii-1]}')
    
        tmp1 = []
        tmp2 = []
        for vol in np.unique(poro_df['volume']):
            porosity_info = poro_df[poro_df["volume"]==vol]['porosity']

            valid_poro = np.logical_and(porosity_info >= lower_bound,\
                                    porosity_info <= upper_bound)
            
            tmp_check = (np.sum(valid_poro) / len(porosity_info))

            tmp1.append(tmp_check >= 0.95)
            tmp2.append(tmp_check)
        
        poro_check[ii] = tmp1
        valid_frac[ii] = tmp2
    
    
    # plt.plot([min(poro_df['volume']), max(poro_df['volume'])], [ep0, ep0],'--k', alpha=0.5)
    plt.xlabel(r"$log_{10} (\frac{V_{B}}{V_{T}})$")
    plt.ylabel("Porosity")
    plt.title(f"REV Porosity : {im_n}")
    plt.legend()

    if ep_thresh != None:
        plt.plot([np.log10(min(poro_df['volume']) / im_size), np.log10(max(poro_df['volume']) / im_size)], 
                 [ep_thresh] * 2, 
                 '--C6',
                 label="Percolation Threshold")

    plt.savefig(f"Figures/REV_Poro_image {im_n[-2:]}.png")

    return poro_check, valid_frac

def analyze_rev_tau(tau_df, ax, actual_tau=None, actual_p=None, ep_thresh=None):
    percents = []
    coeffs = []
    sigma_sqs = []
    p_vals = []
    rsquare = []
    
    for block_size in np.array(block_size_universal)**3:
    # for block_size in np.unique(tau_df["volume"]):
        data_rev = tau_df[tau_df["volume"]==block_size]
        
        x_in = np.array(data_rev['porosity_perc'])
        y_in = data_rev['tau']

        mask = data_rev['tau']!=np.inf
        mask2 = x_in < poro_avg * 1.1
        mask3 = x_in > poro_avg * 0.9

        valid = sum(mask2 * mask3)
        percent = valid / x_in.size * 100
        percents.append(percent)
        
        if all_mask:
            x_in = x_in[mask * mask2 * mask3]
            y_in = y_in[mask * mask2 * mask3]
        
        else:
            x_in = x_in[mask]
            y_in = y_in[mask]

        def loss(params):
            # tomadachis - sotirchos, eqn13 
            # https://sci-hub.se/https://aiche.onlinelibrary.wiley.com/doi/10.1002/aic.690390304
            b = params
            y_pred = func(x_in, b)
            # y_pred = ((1 - a) / (x_in - a)) ** b

            return np.mean((y_in - y_pred)**2)

        guess = fmin(loss, [0.872], disp=False)
        b = guess
        y_pred = func(x_in, b)
        clear_nan = np.array(~np.isnan(y_pred))

        xrange = np.linspace(ep0,1,1000)
        # xrange = np.unique(x_in)
        y_line = func(xrange, b)
    
        # r2 = r2_score(y_in[clear_nan], y_pred[clear_nan])
        residuals = y_in - y_pred
        sigma_sq = np.sum(residuals ** 2) / (len(y_in) - 2)

        chi2_stat = np.sum(residuals**2) / sigma_sq
        ddof = len(y_in) - 2 #2 parameters
        p_value = 1 - chi2.cdf(chi2_stat, ddof)

        sigma_sqs.append(sigma_sq)
        coeffs.append(guess)
        p_vals.append(p_value)
        # rsquare.append(r2)

    x_vals = np.linspace(ep0, 1, 1000)
    t_s_colors = [cmap(i / len(coeffs)) for i in range(len(coeffs))]

    plt.sca(ax)

    for ind, (b, size) in enumerate(zip(coeffs, np.unique(tau_df["volume"]))):
        # print(b[0])
        y_vals = func(x_vals, b)
        # print(y_vals)
        plt.plot(x_vals, y_vals, '--', color=t_s_colors[ind], label=f"{round(size**(1/3))}")

        plt.xlabel("Porosity")
        plt.ylabel(r"$\tau$")
        plt.title(fr"$\tau$ vs Porosity : {im_n[-2:]}")

        plt.ylim([1, 10])
    
    if actual_p != None and actual_tau != None:
        plt.plot(actual_p, actual_tau, '.k', label="Ground Truth")

    if ep_thresh != None:
        plt.plot([ep_thresh] * 2, [0, 10], '--k', alpha=0.5, label="Percolation Threshold")

    
    plt.legend()

    return percents, coeffs, sigma_sqs, p_vals

all_coeffs = {}
all_sigma_sq = {}
all_p_vals = {}
all_rsquare = {}
all_sizes = {}
all_poro_check = {}
all_tau_check = {}
all_TS_check = {}
all_tau_comparison = {}
all_vol_fracs = {}
all_unique_vol = {}
all_tau_fracs = {}
all_rev_vols = {}
all_im_size = {}
all_tau_valid = {}


image_names = glob("Blunt_REV_set_blocks/*.csv")

image_names = [f[-12:-4] for f in image_names]
image_names.sort()

files = glob("Blunt/rev*.xlsx")
files.sort()

# change this to modify which images to process
for im_n in image_names[:]:
    print(im_n)
    with h5py.File("Blunt_images.h5") as f:
        im = np.array(f[f"{im_n}"])
        p = im.sum() / im.size
        temp = np.array([np.array(f["tortuosity_x"][f"{im_n}"]),
                         np.array(f["tortuosity_y"][f"{im_n}"]),
                         np.array(f["tortuosity_z"][f"{im_n}"]),]).flatten()
    
    rev_data = pd.read_csv(f"Blunt/csv_files/rev_{im_n[-2:]}.raw.csv")
    data_ep = pd.read_csv(f"porosity_threshold_Blunt/{im_n[-12:]}.csv")
    set_blocks_data = pd.read_csv(f"Blunt_REV_all_same_blocks/{im_n}.csv")
    # tau_data = pd.read_csv(f"Blunt/csv_files/rev_{im_n[-2:]}.raw.csv")
    tau_data = pd.read_csv(f"Blunt_REV_all_same_blocks/{im_n}.csv")
    porosity_data = pd.read_csv(f"Blunt_REV_porosity/{im_n}.csv")
    ep0 = np.array(data_ep[data_ep['axis']==0]['eps_thresh'])[0]
    tmp_taus = pd.read_csv(f"tmp.csv")

    actual_tau = np.array(tmp_taus[tmp_taus["name"]==im_n]).flatten()[2:]

    def func(x_in, b):
        return ((1 - ep0) / (x_in - ep0)) ** b
    

    # PLOTTING
    macrofig = plt.figure(dpi=300, figsize=[10,7])
    # plt.tight_layout(pad=2.0)
    gs = gridspec.GridSpec(2,6, height_ratios=[1, 1])
    ax1 = macrofig.add_subplot(gs[0,0:3])
    ax2 = macrofig.add_subplot(gs[0,3:6])
    ax3 = macrofig.add_subplot(gs[1,0:2])
    ax4 = macrofig.add_subplot(gs[1,2:4])
    ax5 = macrofig.add_subplot(gs[1,4:6])

    plt.subplots_adjust(hspace=0.4)
    plt.subplots_adjust(wspace=1)

    # REV POROSITY GRAPH
    std = np.std(porosity_data['porosity'])
    poro_avg = np.mean(porosity_data['porosity'])

    # poro_fig, poro_ax = plt.subplots(dpi=300)

    poro_check, valid_frac = analyze_rev_poro(porosity_data, 
                                              p_compare=p, 
                                              im_n=im_n[-2:], 
                                              im_size=im.size,
                                              ax=ax1, 
                                              ep_thresh=ep0)

    # CALCULATIONS FOR T-S GRAPHS

    # fig_ts, ax_ts = plt.subplots(dpi=300)
    percents, coeffs, sigma_sqs, p_vals, = analyze_rev_tau(set_blocks_data, ax2, 
                                                           actual_tau=actual_tau[0], 
                                                           actual_p=p,
                                                           ep_thresh=ep0)


    all_coeffs[im_n] = coeffs
    all_sigma_sq[im_n] = sigma_sqs
    all_p_vals[im_n] = p_vals
    all_sizes[im_n] = np.unique(tau_data["volume"])
    all_poro_check[im_n] = poro_check
    all_vol_fracs[im_n] = valid_frac
    all_unique_vol[im_n] = np.unique(porosity_data['volume'])
    # all_rsquare[im_n] = r_sq

    # ==========TS CHECK==========
    predicted_tau = func(p,all_coeffs[im_n])

    ts_check = 1 - (actual_tau[1] - predicted_tau) / actual_tau[1]
    all_TS_check[im_n] = ts_check

    # ==========TAU CHECK==========
    # filtering data to only include valid points, between +-5, 10, 15, 20% of actual tau

    tau_check = {}
    tau_fracs = {}
    for jj in range(1,5):
        upper_bound = p * (1 + jj * 0.05)
        lower_bound = p * (1 - jj * 0.05)
        
        tmp1 = []
        tmp2 = []

        for vol in np.unique(tau_data['volume']):
            porosity_info = tau_data[tau_data['volume']==vol]['porosity_perc']
            poro_mask = np.logical_and(porosity_info >= lower_bound,\
                                      porosity_info <= upper_bound)
            
            tau_info = 1 / np.array((tau_data[tau_data['volume']==vol]['tau']))[poro_mask]
            # average_tau = np.mean(tau_info)
            average_tau = 1 / actual_tau[1]

            # boundaries based on average tau within specific bin
            upper_bound_tau = average_tau * (1 + jj * 0.05)
            lower_bound_tau = average_tau * (1 - jj * 0.05)

            valid_tau = np.logical_and(tau_info >= lower_bound_tau,\
                                       tau_info <= upper_bound_tau)
            
            
            tmp_check = (np.sum(valid_tau) / len(tau_info))
            
            tmp1.append(tmp_check > 0.95)
            tmp2.append(tmp_check)
        
        tau_check[jj] = tmp1
        tau_fracs[jj] = tmp2

    all_tau_check[im_n] = tau_check
    all_tau_fracs[im_n] = tau_fracs
    all_rev_vols[im_n] = np.unique(tau_data['volume'])
    all_im_size[im_n] = np.prod(im.shape)

    # VIOLIN PLOTS
    try:
        fig_t, plots, mean_taus, standard_devs, tau_inv, valid_tau_values = rev_plot(
            tau_data,
            # rev_data,
            im_n=im_n[-2:],
            size=im.shape[0], 
            title=im_n[-2:],
            ep_thresh=poro_avg,
            figs=macrofig,
            axes=[ax3,ax4,ax5,ax1],
            bounds=True,
            actual_tau=actual_tau)
    
    except:
        fig_t, plots, mean_taus, standard_devs, tau_inv, valid_tau_values = rev_plot(
            # set_blocks_data,
            rev_data,
            im_n=im_n[-2:],
            size=im.shape[0], 
            title=im_n[-2:],
            ep_thresh=poro_avg,
            figs=macrofig,
            axes=[ax3,ax4,ax5,ax1],
            bounds=True,
            actual_tau=actual_tau)
    
    all_tau_valid[im_n] = valid_tau_values

    # hex_plot(rev_data, 
    #          im.shape[0], 
    #          ep0, 
    #         #  bounds=True, 
    #         #  axes=[ax3,ax4,ax5,ax1], 
    #          actual_tau=actual_tau)
    # std_tau = [np.std(group) for group in tau_inv]
    # tau_mask = [np.logical_and(group>=((1/actual_tau)-(2*s)), 
    #                            group<=((1/actual_tau)+(2*s))) 
    #                            for (group, s) in zip(tau_inv, std_tau)]
    # all_tau_check[im_n] = [sum(group[mask])/len(group) for (group, mask) in zip(tau_inv, tau_mask)]
    # all_tau_check[im_n] = [sum(np.array(mean_taus[i:]) - actual_tau)**2 for i in range(len(mean_taus) - 1)]
    # all_tau_check[im_n] = [(i-j)/i <= 0.1 for i, j in zip(mean_taus, mean_taus[1:])]
    # all_tau_check[im_n] = np.array(standard_devs) / poro_avg <= 0.1

    # all_tau_comparison[im_n] = [actual_tau, predicted_tau]

    # if all_mask:
    #     plt.savefig(f"Figures/Macro_{im_n}_masked")
    
    # else:
    #     plt.savefig(f"Figures/Macro_{im_n}_no_mask")

    plt.savefig(f"Figures/Macro_Final_{im_n}.png")

    

In [ ]:
# all_coeffs.keys()
(all_rev_vols["image A1"])**(1/3)
# len(np.array([all_vol_fracs[k][spread] for spread in all_vol_fracs[k].keys()]).flatten())

all_tau_valid["image A1"].keys()

ts_check

In [ ]:
tmp = []

PLOT_A = False
PLOT_B = False

t_s_colors = [cmap(i / len(all_coeffs[im_n])) for i in range(len(all_coeffs[im_n]))]

# all_vol_fracs[]

for i, k in enumerate(all_coeffs.keys()):

    # fig, ax = plt.subplots(dpi=300)
    # plt.sca(ax)
    # plt.plot(np.log10(all_sizes[k]), all_coeffs[k], '.-')
    # plt.ylim([0,1.5])
    # plt.title(f"Coeffs vs Block Size : {k}")
    tmp.append({
        "im_name" : k,
        # "porosity_check" : True in (np.array([all_vol_fracs[k][spread] for spread in all_vol_fracs[k].keys()]).flatten() > 0.95),
        # "tau_check" : True in (np.round(np.array([all_tau_fracs[k][spread] for spread in all_tau_fracs[k].keys()]).flatten(), 5) > 0.8),
        # "ts_check" : True in (all_TS_check[k] <= 0.1),
        "porosity_check_value" : max(np.round(np.array([all_vol_fracs[k][spread] for spread in all_vol_fracs[k].keys()]).flatten(), 5)),
        "tau_check_value" : max(np.round(np.array([all_tau_fracs[k][spread] for spread in all_tau_fracs[k].keys()]).flatten(), 5)),
        "ts_check_value" : [0 if (i>=1.5 or i<=0) else i for i in np.round((np.array(all_TS_check[k]).flatten()), 4) ],

    })

    subplot_size = 4
    if PLOT_A and PLOT_B:
        fig, ax = plt.subplots(ncols=2, dpi=300, figsize=(2 * subplot_size, subplot_size))
        plt.subplots_adjust(hspace=1)

        plt.sca(ax[0])
        # ax[0].set_aspect('equal', adjustable='box')
        for spread in all_vol_fracs[k].keys():
            plt.plot(np.log10(all_unique_vol[k] / all_im_size[k]), all_vol_fracs[k][spread], '.', label=rf"$\pm${spread*0.05:.2f}")
        plt.plot([-5, -1.3], [0.8, 0.8], '--k', alpha=0.5)
        plt.xlabel(r"$log_{10} (\frac{V_{B}}{V_{T}})$")
        plt.ylabel("f")
        plt.xlim([-5, -1.3])
        plt.ylim([-0.1, 1.1])
        plt.title(f"Valid Porosity Fraction vs Volume : {k[-2:]}")
        plt.legend()
        # plt.savefig(fr"Figures/Volume_fraction {k[-2:]}")

        plt.sca(ax[1])
        # ax[1].set_aspect('equal', adjustable='box')
        
        # for spread in all_tau_fracs[k].keys():
        for spread in all_tau_valid[k][0].keys():
            plt.plot(np.log10(all_rev_vols[k] / all_im_size[k]), all_tau_valid[k][0][spread], '.', label=rf"$\pm${spread*0.05:.2f}")
        plt.plot([-5, -1.3], [0.8, 0.8], '--k', alpha=0.5)
        plt.xlabel(r"$log_{10}(\frac{V_{B}}{V_{T}})$")
        plt.ylabel("f")
        plt.xlim([-5, -1.3])
        plt.ylim([-0.1, 1.1])
        plt.title(f"Valid Tau Fraction vs Volume : {k[-2:]}")
        plt.legend()

        plt.tight_layout()
        plt.savefig(fr"Figures/Macro_Fraction_image {k[-2:]}")

    elif PLOT_A:
        fig, ax = plt.subplots(dpi=300)

        for spread in all_vol_fracs[k].keys():
            plt.plot(all_unique_vol[k], all_vol_fracs[k][spread], '.', label=rf"$\pm${spread*0.05:.2f}")
        plt.xlabel(r"$log_{10}(V_{B})$")
        plt.ylabel("f")
        plt.ylim([0, 1.1])
        plt.title(f"Valid Porosity Fraction vs Volume : {k[-2:]}")
        plt.legend()
        plt.savefig(fr"Figures/Volume_fraction {k[-2:]}")

    elif PLOT_B:
        fig, ax = plt.subplots(dpi=300)

        for spread in all_tau_fracs[k].keys():
            plt.plot(all_rev_vols[k], all_tau_fracs[k][spread], '.', label=rf"$\pm${spread*0.05:.2f}")
        plt.xlabel(r"$log_{10}(V_{B})$")
        plt.ylabel("f")
        plt.ylim([0, 1.1])
        plt.title(f"Valid Tau Fraction vs Volume : {k[-2:]}")
        plt.legend()
        plt.savefig(fr"Figures/Tau_fraction_average_tau {k[-2:]}")

fig_tau_compare, ax_tau_compare = plt.subplots(dpi=300)

def func(x_in, b, ep0):
    return ((1 - ep0) / (x_in - ep0)) ** b

for i, k in enumerate(all_coeffs.keys()):
    b = all_coeffs[k][-1]
    with h5py.File("Blunt_images.h5") as f:
        im = np.array(f[f"{im_n}"])
        p = im.sum() / im.size

    data_ep = pd.read_csv(f"porosity_threshold_Blunt/{im_n[-12:]}.csv")
    ep0 = np.array(data_ep[data_ep['axis']==0]['eps_thresh'])[0]
    actual_tau = np.array(tmp_taus[tmp_taus["name"]==k]).flatten()[2:][0]
    pred_tau = func(x_in=p, b=b, ep0=ep0)

    if "S2" in k:
        plt.text(actual_tau+0.05, pred_tau-0.4, k[-2:])
    
    elif "S6" in k or "S7" in k:
        plt.text(actual_tau-0.1, pred_tau+0.09, k[-2:])

    elif actual_tau<=10 and pred_tau<=10:
        plt.text(actual_tau+0.05, pred_tau+0.05, k[-2:])
    
    plt.plot(actual_tau, pred_tau, '.')

plt.plot([1,10], [1,10], '--k')
plt.ylim([1,10])
plt.xlim([1,10])
plt.xlabel(r"$\tau$ Actual")
plt.ylabel(r"$\tau$ Predicted")
plt.title(r"Prediction vs Actual $\tau$")
# plt.legend()
tmp2 = pd.DataFrame(tmp)
tmp2

fig_tau_error, ax_tau_error = plt.subplots(dpi=300)

error_val = []


shortened_names = [f[-2:] for f in image_names]
for i, k in enumerate(all_coeffs.keys()):
    ts_check_value = [0 if (i>=1.5 or i<=0) else i for i in np.round((np.array(all_TS_check[k]).flatten()), 4) ]
    print(ts_check_value)
    error_val.append(ts_check_value[-1])

plt.bar(np.arange(len(all_coeffs.keys())), np.abs(np.array(error_val)-1) * 100)
plt.xticks(np.arange(len(all_coeffs.keys())), shortened_names)
plt.xlabel("Image Identity")
plt.ylabel("Percent Error (%)")
plt.title("Percent Error in Tortuosity Prediction")

# for i in tmp2['tau_check_value']:
#     print(i)

# print(tmp2)

In [ ]:
shortened_names = [f[-2:] for f in image_names]
print(shortened_names)

In [ ]:
all_coeffs["image A1"][-1]

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.animation import ArtistAnimation, PillowWriter, FuncAnimation

image_names = glob("Blunt_REV_set_blocks/*.csv")
data = pd.read_hdf("psd_data_local_thickness.h5", key="blunt_data")
porosity_data = pd.read_csv("porosity_threshold_Blunt/all_data.csv")

image_names = [f[-12:-4] for f in image_names]
image_names.sort()

func = lambda y, a, dx, dy : a * (y - dy)**2 + dx

y_vals = np.linspace(2.5, 5.5, 100)
x_vals = func(y_vals, -5, 14, 4)

thresholds = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
interpolation_points = [0.1]
# thresholds = [0.8]
# interpolation_points = [0.1, 0.2, 0.3, 0.4]

# frames_data = []

x_set = []
y_set = []
colour_set = []
t = []

fig, ax = plt.subplots(dpi=300)

for threshold in thresholds:
    for interpolation_point in interpolation_points:
    # parabola = plt.plot(x_vals, y_vals, '--k')

        x_plot_points = []
        y_plot_points = []
        colours = []
        with h5py.File("Blunt_images.h5") as f:
            for im_n in image_names:
                im = np.array(f[f"{im_n}"])
                p = im.sum() / im.size
                data_ep = pd.read_csv(f"porosity_threshold_Blunt/{im_n[-12:]}.csv")

                mask1 = (porosity_data['name']==im_n) & (porosity_data['axis']==0)
                ep01 = np.array((porosity_data[mask1]['eps_thresh_perc']))[0]
                ep0 = np.array(data_ep[data_ep['axis']==0]['eps_thresh'])[0]

                # print(im_n,ep01,ep0)
                df = data[im_n]

                check1 = tmp2[tmp2['im_name'] == im_n]['porosity_check_value']
                check2 = tmp2[tmp2['im_name'] == im_n]['tau_check_value']

                closeness = sum(check1 + check2) / 2
                # print(im_n, closeness)

                # plt.plot((df.data_bins), df.data_cdf)

                x_interp = np.interp(interpolation_point, df.data_cdf[::-1], df.data_bins[::-1])
                # print(df.data_cdf)
                y = p / ep0

                x_plot_points.append(x_interp)
                y_plot_points.append(y)

                if np.all(check1 > threshold) and np.all(check2 > threshold):
                    colours.append("green")
                    # plot_points = plt.plot(x_interp+0.06, y+0.06, '.g', label=im_n, )
                    # plt.scatter(x_interp+0.06, y+0.06, label=im_n, c=closeness, cmap="viridis", norm=norm)
                
                else:
                    colours.append("red")
                    # plot_points = plt.plot(x_interp, y, '.r', label=im_n)
                    # plt.text(x_interp+0.06, y+0.06, im_n[-2:],)

                # plt.text(x_interp+0.06, y+0.06, im_n[-2:],)

                # frames_data.append(plot_points)
                # print(im_n, x_interp, y)
        
        # plt.savefig(f"Figures/PSD_analysis - T={threshold}.png")
        # frames_data.append([plot_points])

        x_set.append(x_plot_points)
        y_set.append(y_plot_points)
        colour_set.append(colours)
        t.append(threshold)

plt.xlabel("Size Value")
plt.ylabel(r"Porosity Ratio $(\frac{\varepsilon}{\varepsilon_{p}})$")
plt.grid(True)
plt.xlim([3, 25])
plt.ylim([0, 6])
scatter_data = plt.scatter(x_set[0], y_set[0], s=5, c=colour_set[0])

labels = [im_n[-2:] for im_n in image_names]

texts = [plt.text(x+0.06, y+0.06, label) for x, y, label in zip(x_set[0], y_set[0], labels)]

def update(frame):
    # plt.title(f"Pore Size Distribution Analysis - I = {np.round(0.1 + frame*0.1, 2)}")
    plt.title(f"Pore Size Distribution Analysis - T = {np.round(0.9 - frame*0.1, 2)}")
    # scatter_data = plt.scatter(x_set[frame], y_set[frame], s=5, c=colour_set[frame])
    scatter_data.set_offsets(np.c_[x_set[frame], y_set[frame]])
    scatter_data.set_color(colour_set[frame])
    for i, txt in enumerate(texts):
        txt.set_position((x_set[frame][i], y_set[frame][i]))
    
    plt.savefig(f"Figures/PSD_analysis - Frame {frame}.png")
    return [scatter_data]

ani = FuncAnimation(fig, update, frames=int(len(thresholds) * len(interpolation_points)), blit=True)
ani.save("test2.gif", writer=PillowWriter(fps=2))
# ani = ArtistAnimation(fig, frames_data, interval=300, blit=True)
# ani.save("test.gif", writer=PillowWriter(fps=3))

    

In [ ]:
import os

files = glob("Blunt/rev*.xlsx")
input_folder = 'Blunt'
output_folder = 'Blunt/csv_files'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith('.xlsx'):
        filepath = os.path.join(input_folder, filename)
        df = pd.read_excel(filepath)  # reads the first sheet by default
        
        csv_name = os.path.splitext(filename)[0] + '.csv'
        csv_path = os.path.join(output_folder, csv_name)
        
        df.to_csv(csv_path, index=False)
        print(f"Converted: {filename} -> {csv_name}")